### See previous notebook for initial thoughts  on what to  predict


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
proteins = pd.read_csv('/content/drive/My Drive/UNH Spring/Protein_seq_project/structID_allchains.csv',index_col=0)

In [3]:
proteins.head()

,struct_ID,all_chain_sequences_as_string,number_chains,classification
0,101M,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,1,OXYGEN TRANSPORT
1,102L,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...,1,HYDROLASE(O-GLYCOSYL)
2,102M,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,1,OXYGEN TRANSPORT
3,103L,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...,1,HYDROLASE(O-GLYCOSYL)
4,103M,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,1,OXYGEN TRANSPORT


Subset the data for viruses only

In [4]:
viruses = [v for v in list(proteins['classification'].unique()) if 'vir' in v.lower()]
viruses

['VIRAL PROTEIN',
 'Viral protein/Immune system',
 'VIRUS',
 'Viral protein/isomerase',
 'COMPLEX (MHC/VIRAL PEPTIDE/RECEPTOR)',
 'ANTIVIRAL PROTEIN',
 'ANTI-HYPERTENSIVE, ANTI-VIRAL PROTEIN',
 'Viral protein/inhibitor',
 'Virus/Receptor',
 'VIRAL PEPTIDE',
 'VIRUS/VIRAL PROTEIN',
 'INFLUENZA VIRUS',
 'VIRUS/VIRAL PROTEIN RECEPTOR',
 'Viral protein, transferase',
 'COMPLEX (SH3 DOMAIN/VIRAL ENHANCER)',
 'Viral protein receptor',
 'Viral protein, hydrolase',
 'ISOMERASE/VIRAL PROTEIN',
 'Viral protein/receptor/Immune system',
 'Immune system/viral protein receptor',
 'Viral protein complex',
 'Viral protein/receptor',
 'Viral protein/ translation',
 'Isomerase/Viral protein',
 'Viral protein inhibitor',
 'Avirulence Protein',
 'Virus/Immune system',
 'ANTIVIRAL PROTEIN, RECOMBINATION',
 'hydrolase/viral protein',
 'Viral protein/hydrolase',
 'VIRAL ADHESION PROTEIN',
 'VIRUS/RECEPTOR',
 'ADENOVIRUS',
 'VIRULENCE FACTOR',
 'VIRUS COAT PROTEIN',
 'VIRUS LIKE PARTICLE',
 'Virus like Partic

In [5]:
proteins = proteins[proteins['classification'].isin(viruses)]
proteins.shape

(3595, 4)

In [6]:
proteins['all_chain_sequences_as_string'].iloc[0]

'ASMTGGQQMGAPITAYAQQTRGLLGCIITSLTGRDKNQVEGEVQIVSTATQTFLATCINGVCWTVYHGAGTRTIASPKGPVIQMYTNVDQDLVGWPAPQGSRSLTPCTCGSSDLYLVTRHADVIPVRRRGDSRGSLLSPRPISYLKGSSGGPLLCPTGHAVGLFRAAVCTRGVAKAVDFIPVENLETTMRGSHHHHHH|ASMTGGQQMGAPITAYAQQTRGLLGCIITSLTGRDKNQVEGEVQIVSTATQTFLATCINGVCWTVYHGAGTRTIASPKGPVIQMYTNVDQDLVGWPAPQGSRSLTPCTCGSSDLYLVTRHADVIPVRRRGDSRGSLLSPRPISYLKGSSGGPLLCPTGHAVGLFRAAVCTRGVAKAVDFIPVENLETTMRGSHHHHHH|KKGSVVIVGRIVLSGKPAIIPKK|KKGSVVIVGRIVLSGKPAIIPKK'

Recall, a chain in a protein is split up with a "|"

Let's put all the viruses in a list

In [0]:
all_viruses = list(proteins['all_chain_sequences_as_string'])[:1400]

In [8]:
len(all_viruses)

1400

We can seperate a unique virus with a "+", and within each unique virus we can seperate them with a "|".

In this interpretation a model can learn chains, but also has the  ability to distinquish  between unique viruses (all classified by their structID)

In [0]:
all_viruses = "+".join(all_viruses)

In [10]:
len(all_viruses)

1070376

In [11]:
all_viruses[:600]

'ASMTGGQQMGAPITAYAQQTRGLLGCIITSLTGRDKNQVEGEVQIVSTATQTFLATCINGVCWTVYHGAGTRTIASPKGPVIQMYTNVDQDLVGWPAPQGSRSLTPCTCGSSDLYLVTRHADVIPVRRRGDSRGSLLSPRPISYLKGSSGGPLLCPTGHAVGLFRAAVCTRGVAKAVDFIPVENLETTMRGSHHHHHH|ASMTGGQQMGAPITAYAQQTRGLLGCIITSLTGRDKNQVEGEVQIVSTATQTFLATCINGVCWTVYHGAGTRTIASPKGPVIQMYTNVDQDLVGWPAPQGSRSLTPCTCGSSDLYLVTRHADVIPVRRRGDSRGSLLSPRPISYLKGSSGGPLLCPTGHAVGLFRAAVCTRGVAKAVDFIPVENLETTMRGSHHHHHH|KKGSVVIVGRIVLSGKPAIIPKK|KKGSVVIVGRIVLSGKPAIIPKK+EVQLQQSGAELVRPGASVKLSCTTSGFNIKDIYIHWVKQRPEQGLEWIGRLDPANGYTKYDPKFQGKATITVDTSSNTAYLHLSSLTSEDTAVYYCDGYYSYYDMDYWGPGTSVTVSSAKTTAPSVYPLAPVCGDTTGSSVTLGCLVKGYFPEP'

Choose a  sliding  window  of  a  certain size (start  with  30 or so). Capture the  ith  window  and then capture the ith letter. Scroll down all viruses

In [0]:
# Length of extracted character sequences
maxlen = 30
# This holds our extracted sequences
sequences = []
# This holds the targets (the follow-up characters)
next_AA = []
for i in range(0,len(all_viruses)-maxlen):
  sequences.append(all_viruses[i:i+maxlen])
  next_AA.append(all_viruses[i+maxlen])

In [13]:
len(sequences),len(next_AA)

(1070346, 1070346)

In [14]:
sequences[0], next_AA[0]

('ASMTGGQQMGAPITAYAQQTRGLLGCIITS', 'L')

In [15]:
all_viruses[:31]

'ASMTGGQQMGAPITAYAQQTRGLLGCIITSL'

In [16]:
chars = sorted(set(all_viruses))
chars

['+',
 'A',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L',
 'M',
 'N',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'V',
 'W',
 'X',
 'Y',
 '|']

In [17]:
chars.index('A')

1

In [0]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [19]:
char_indices

{'+': 0,
 'A': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'K': 9,
 'L': 10,
 'M': 11,
 'N': 12,
 'P': 13,
 'Q': 14,
 'R': 15,
 'S': 16,
 'T': 17,
 'V': 18,
 'W': 19,
 'X': 20,
 'Y': 21,
 '|': 22}

One hot them

In [0]:
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

In [21]:
x.shape, y.shape

((1070346, 30, 23), (1070346, 23))

In [0]:
for i, sequence in enumerate(sequences):
  for t,char in enumerate(sequence):
    x[i,t,char_indices[char]] = 1
  y[i,char_indices[next_AA[i]]] =1

In [0]:
x[0,:,:]

In [0]:
y[0,:]

In [0]:
!pip install tensorflow==1.14.0

## Load in virus model and start sequence prediction

In [0]:
import keras
from keras.models import load_model

In [28]:
model = load_model("/content/drive/My Drive/UNH Spring/Protein_seq_project/Create_Virus_Model/VirusBestLoss_1million.h5")
model.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 130)               60060     
_________________________________________________________________
dense_4 (Dense)              (None, 23)                3013      
Total params: 63,073
Trainable params: 63,073
Non-trainable params: 0
_________________________________________________________________


In [0]:
#helper functions
def encode_sequence(string):
  string_encoded = x = np.zeros((maxlen, len(chars)), dtype=np.bool)
  for i,j in enumerate(string):
    #look up jth letter
    string_encoded[:,chars.index(j)]  = 1
  return(string_encoded)

def sample(preds, temperature=.9):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds.ravel(), 1)
    return np.argmax(probas)



In [0]:
starting_seq = all_viruses[:30]
length_to_predict = 1000
results = np.zeros((1000),dtype='object')
for i,j in enumerate(starting_seq):
  results[i] = j
for i in range(0,len(results)-maxlen-1):
  #grab the letters
  letters = "".join(results[i:i+maxlen-1].tolist())
  X = encode_sequence(letters)
  pred = model.predict(X.reshape((1,30,23)))
  next_letter = chars[sample(pred)]
  #add to results
  results[i+maxlen] = next_letter






In [0]:
results

Generated  Sequence

In [128]:
"".join(results.tolist()[:-1])

'ASMTGGQQMGAPITAYAQQTRGLLGCIITSADDLLSLAVVAAASLSVATMSYYAAREEHRKKKKAAKSAKKASKAKAIIAG+NKVVVVVVGGVQQQTTMYLLFLGGASEGKLIDNNNNNNNNNTIIQNQKSSQFFERQKKGGGSGAGTGGIGGTIGTGGIIIIIIGITTTATKLPSEMFWSQDIINDIQVNNNKQNQQQQPQKDKKKKTIFQSAKEEEEEFFVFEEEEAEEEEEGGPRIIGDGIVKGVKFPYIFVFFFFIIADGAAAAAAAAAAAAAAAAAAAAASSAMKLCRRLVLRVRGYRQRSISAASSQAIGSAIAAADDAYGAATTLLRSYYSNYKLFSQAEDTFFFFFVLIIRMNKKRRPPQKKRQSQKQSTGADIQNNGFFLGKKKGGKKGDDAEQVQRRKKGKGYRIDNNKVTKKKKKKKTKTKQQSPLLAGFLQQQIDDKSSSSSSSSSSSSSSSSSGKGKRSSISSTTTTVTSSSSSSSSPAQQLLTQGKGGDYLFFGLLSVVLLLLLLLLLRRDDDDDDAALMATNNKSQQQIQQQQQRQQQQQQQQQQQQQQYMKAIIQQQQHQ||SSNSSSSASASSSASSSSASSSSSSSSASSVIVNNEERRKRVRQQRKLLRRRRRRRRRRRRRRRRRRRRRRREEKRRRNMYVIKKKKKKKKKKKKKKKKKKKKKILQEPNGGIIISIVVVVVVVVVEIIVVVVVVVITKNATMQQAAAANAANARARQQLLQQQQQQIIKQQRRRRRKRRKRKRRKSIGAGIIIIIVIDIYIQADDADNDYQQQTPFKSQIIDIIIPIIIIIIIIIVTPDDQADDDDEPYYYEEYEEPSEYYYYYYKIFVIDDLDEQQDDDDTDDDDDDNNKKKKPRPGQKVVTYLILIITPIIKIVIIGGGGGGGGSGGAAAQQYYGDNFFADDLGESFLFFVFRRKKFIITKKKMRYYNNNNKNQNNQPPLAAIIIIIDNETIKKKEFMIIIIISSSSSKRARRQIIQQ

Test Sequence

In [129]:
all_viruses[:1000]

'ASMTGGQQMGAPITAYAQQTRGLLGCIITSLTGRDKNQVEGEVQIVSTATQTFLATCINGVCWTVYHGAGTRTIASPKGPVIQMYTNVDQDLVGWPAPQGSRSLTPCTCGSSDLYLVTRHADVIPVRRRGDSRGSLLSPRPISYLKGSSGGPLLCPTGHAVGLFRAAVCTRGVAKAVDFIPVENLETTMRGSHHHHHH|ASMTGGQQMGAPITAYAQQTRGLLGCIITSLTGRDKNQVEGEVQIVSTATQTFLATCINGVCWTVYHGAGTRTIASPKGPVIQMYTNVDQDLVGWPAPQGSRSLTPCTCGSSDLYLVTRHADVIPVRRRGDSRGSLLSPRPISYLKGSSGGPLLCPTGHAVGLFRAAVCTRGVAKAVDFIPVENLETTMRGSHHHHHH|KKGSVVIVGRIVLSGKPAIIPKK|KKGSVVIVGRIVLSGKPAIIPKK+EVQLQQSGAELVRPGASVKLSCTTSGFNIKDIYIHWVKQRPEQGLEWIGRLDPANGYTKYDPKFQGKATITVDTSSNTAYLHLSSLTSEDTAVYYCDGYYSYYDMDYWGPGTSVTVSSAKTTAPSVYPLAPVCGDTTGSSVTLGCLVKGYFPEPVTLTWNSGSLSSGVHTFPAVLQSDLYTLSSSVTVTSSTWPSQSITCNVAHPASSTKVDKKIEPR|DIVMTQSPSSLTVTTGEKVTMTCKSSQSLLNSRTQKNYLTWYQQKPGQSPKLLIYWASTRESGVPDRFTGSGSGTDFTLSISGVQAEDLAVYYCQNNYNYPLTFGAGTKLELKRADAAPTVSIFPPSSEQLTSGGASVVCFLNNFYPKDINVKWKIDGSERQNGVLNSWTDQDSKDSTYSMSSTLTLTKDEYERHNSYTCEATHKTSTSPIVKSFNRNEC|VKAETRLNPDLQPTEX+MSPTSILDIRQGPKEPFRDYVDRFYKTLRAEQASQEVKNWMTETLLVQNANPDCKTILKALGPGATLEEMMTACQGVGGPGHKARVL+AVTVVPDP

It looks like the model didn't really get the sequence.
But it did make a few protein chains and even a full protein! Marked with a '+'



In [131]:
## comparing simialrity
a = "".join(results.tolist()[:-1])
b = all_viruses[:1000]

import difflib
seq=difflib.SequenceMatcher(a=a.lower(), b=b.lower())
seq.ratio()

0.03301650825412707

Anything after this involved training the sequential model

Load in model again after it crashed and continue training

In [0]:
import keras
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
file_path_best_loss = "/content/drive/My Drive/UNH Spring/Protein_seq_project/Create_Virus_Model/VirusBestLoss.h5"
file_path_best_acc = "/content/drive/My Drive/UNH Spring/Protein_seq_project/Create_Virus_Model/VirusBestAcc.h5"

checkpoint_loss = ModelCheckpoint(file_path_best_loss, 
                              monitor='loss', 
                              verbose=1, 
                              save_best_only=True, 
                              mode='min')
checkpoint_acc = ModelCheckpoint(file_path_best_acc, 
                              monitor='accuracy', 
                              verbose=1, 
                              save_best_only=True, 
                              mode='max')

callbacks_list = [checkpoint_loss,checkpoint_acc]

model = load_model(file_path_best_loss, compile=False)
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_9 (GRU)                  (None, 100)               37200     
_________________________________________________________________
dense_11 (Dense)             (None, 23)                2323      
Total params: 39,523
Trainable params: 39,523
Non-trainable params: 0
_________________________________________________________________


In [0]:
iterations = 5
for i in range(iterations):
  model.fit(x, y,
            validation_split=0.1,
            batch_size=128,
            epochs=10,
            callbacks=callbacks_list)
  pd.DataFrame(model.history.history).to_csv("/content/drive/My Drive/UNH Spring/Protein_seq_project/Create_Virus_Model"+"/VirusModelHistory_"+str(i+6)+"_of_10.csv")


In [0]:
import keras
from keras import layers
from keras.callbacks import ModelCheckpoint

model = keras.models.Sequential()
model.add(layers.GRU(130, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
optimizer = keras.optimizers.Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer,
              metrics=['accuracy'])


file_path_best_loss = "/content/drive/My Drive/UNH Spring/Protein_seq_project/Create_Virus_Model/VirusBestLoss_1million.h5"
file_path_best_acc = "/content/drive/My Drive/UNH Spring/Protein_seq_project/Create_Virus_Model/VirusBestAcc_1milion.h5"

checkpoint_loss = ModelCheckpoint(file_path_best_loss, 
                              monitor='loss', 
                              verbose=1, 
                              save_best_only=True, 
                              mode='min')
checkpoint_acc = ModelCheckpoint(file_path_best_acc, 
                              monitor='accuracy', 
                              verbose=1, 
                              save_best_only=True, 
                              mode='max')

callbacks_list = [checkpoint_loss,checkpoint_acc]


model.summary()



Start here for using a smaller sample of viruses

In [0]:
model = load_model(file_path_best_loss)
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_4 (GRU)                  (None, 130)               60060     
_________________________________________________________________
dense_4 (Dense)              (None, 23)                3013      
Total params: 63,073
Trainable params: 63,073
Non-trainable params: 0
_________________________________________________________________


In [0]:
iterations = 7
for i in range(iterations):
  model.fit(x, y,
            validation_split=0.1,
            batch_size=128,
            epochs=10,
            callbacks=callbacks_list)
  pd.DataFrame(model.history.history).to_csv("/content/drive/My Drive/UNH Spring/Protein_seq_project/Create_Virus_Model"+"/1_million_VirusModelHistory_"+str(i+9)+"_of_10.csv")


Train on 963311 samples, validate on 107035 samples
Epoch 1/10
963311/963311 [==============================] - 671s 696us/step - loss: 1.6240 - accuracy: 0.5323 - val_loss: 2.9623 - val_accuracy: 0.2058

Epoch 00001: loss did not improve from 1.62337

Epoch 00001: accuracy did not improve from 0.53329
Epoch 2/10
963311/963311 [==============================] - 657s 683us/step - loss: 1.6218 - accuracy: 0.5334 - val_loss: 2.9632 - val_accuracy: 0.2058

Epoch 00002: loss improved from 1.62337 to 1.62180, saving model to /content/drive/My Drive/UNH Spring/Protein_seq_project/Create_Virus_Model/VirusBestLoss_1million.h5

Epoch 00002: accuracy improved from 0.53329 to 0.53342, saving model to /content/drive/My Drive/UNH Spring/Protein_seq_project/Create_Virus_Model/VirusBestAcc_1milion.h5
Epoch 3/10
963311/963311 [==============================] - 666s 691us/step - loss: 1.6232 - accuracy: 0.5328 - val_loss: 3.0115 - val_accuracy: 0.2029

Epoch 00003: loss did not improve from 1.62180

Epo